In [1]:
nombre = "\n Ricardo Madrigal Urencio \n"
print(nombre)


 Ricardo Madrigal Urencio 



In [2]:
#Todas las librerias a Importar
from pyspark.sql.functions import col, sum, to_date, to_timestamp, regexp_replace
from pyspark.sql.functions import date_format
from pyspark.sql.types import IntegerType, DoubleType, FloatType
from pyspark.sql.functions import hour, minute, second, year, month, dayofmonth, weekofyear
from pyspark.sql.functions import monotonically_increasing_id  

In [3]:
#Crear un solo Dataframe con todos los meses
dir_hdfs = "hdfs://namenode:9000/tmp/amd/ecobici/2019"

print(nombre)

df = spark.read.csv(dir_hdfs, header=True, inferSchema=True)
df.printSchema()



 Ricardo Madrigal Urencio 

root
 |-- Genero_Usuario: string (nullable = true)
 |-- Edad_Usuario: string (nullable = true)
 |-- Bici: string (nullable = true)
 |-- Ciclo_Estacion_Retiro: string (nullable = true)
 |-- Fecha_Retiro: string (nullable = true)
 |-- Hora_Retiro: string (nullable = true)
 |-- Ciclo_Estacion_Arribo: string (nullable = true)
 |-- Fecha_Arribo: string (nullable = true)
 |-- Hora_Arribo: string (nullable = true)
 |-- _c9: timestamp (nullable = true)



In [4]:
print(nombre)

print(df.count())


 Ricardo Madrigal Urencio 

7718156


In [5]:
print(nombre)
df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).show()


 Ricardo Madrigal Urencio 



ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt

KeyboardInterrupt



In [ ]:
print(nombre)

df.filter("_c9 is not null").show(truncate = False)

In [ ]:
#Eliminar Registro Inválido

print (nombre)
print(df.count())


df = df.filter("_c9 is null")
df = df.filter(col("Fecha_Retiro").like('%2019'))

print(df.count())

In [ ]:
print(nombre)

df.orderBy("Fecha_Arribo", ascending = True).show(10, truncate = False)

In [ ]:
#Limpieza del esquema
print(nombre)

datos = df\
.withColumn("Edad_Usuario", col("Edad_Usuario").cast(FloatType())) \
.withColumn("Bici", col("Bici").cast(IntegerType())) \
.withColumn("Ciclo_Estacion_Retiro", col("Ciclo_Estacion_Retiro").cast(IntegerType())) \
.withColumn("Fecha_Retiro", to_date(col("Fecha_Retiro"), "dd/MM/yyyy")) \
.withColumn("Hora_Retiro", date_format(col("Hora_Retiro"), "HH:mm:ss")) \
.withColumn("Ciclo_Estacion_Arribo", col("Ciclo_Estacion_Arribo").cast(IntegerType())) \
.withColumn("Fecha_Arribo", to_date(col("Fecha_Arribo"), "dd/MM/yyyy")) \
.withColumn("Hora_Arribo", date_format(col("Hora_Arribo"), "HH:mm:ss"))\
.withColumn( "id", monotonically_increasing_id().cast(IntegerType()))
datos.printSchema()
datos.orderBy("Fecha_Arribo", ascending = True).show(10, truncate = False)

In [ ]:
#Agregar campos año, mes, día, hora, minuto, segundo, día de la semana
##Separar Fechas y Horas de Retiro en Campos Individuales

from pyspark.sql.functions import hour, minute, second, year, month, dayofmonth, weekofyear
colfs = "Fecha_Retiro"
colhs = "Hora_Retiro"

datos_tiempo_retiro = datos.select(
    col("id"), col("Genero_Usuario"), col("Edad_Usuario"), col("Bici"), col("Ciclo_Estacion_Retiro"),col("Fecha_Retiro"),
    date_format(col(colfs), "EEEE").alias("dia_sem_retiro"),
    dayofmonth(col(colfs)).alias("num_dia_retiro"),
    month(col(colfs)).alias("num_mes_retiro"),
    date_format(col(colfs), "MMMM").alias("mes_retiro"),
    year(col(colfs)).alias("anio_retiro"),
    col("Hora_Retiro"),
    hour(col(colhs)).alias("num_hora_retiro"),
    minute(col(colhs)).alias("minuto_retiro"),    
    second(col(colhs)).alias("segundo_retiro")
    
)

print(nombre)

datos_tiempo_retiro.printSchema()

datos_tiempo_retiro.orderBy('anio_retiro', ascending = True).show(5)

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: reentrant call inside <_io.BufferedReader name=48>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip

In [ ]:
##Separar Fechas y Horas de Arribo en Campos Individuales

from pyspark.sql.functions import hour, minute, second, year, month, dayofmonth, weekofyear
colfs = "Fecha_Arribo"
colhs = "Hora_Arribo"

datos_tiempo_arribo = datos.select(
   col("id"), col("Genero_Usuario"), col("Edad_Usuario"), col("Bici"), col("Ciclo_Estacion_Arribo"), col("Fecha_Arribo"),
    date_format(col(colfs), "EEEE").alias("dia_sem_arribo"),
    dayofmonth(col(colfs)).alias("num_dia_arribo"),
    month(col(colfs)).alias("num_mes_arribo"),
    date_format(col(colfs), "MMMM").alias("mes_arribo"),
    year(col(colfs)).alias("anio_arribo"),
    col("Hora_Arribo"),
    hour(col(colhs)).alias("num_hora_arribo"),
    minute(col(colhs)).alias("minuto_arribo"),    
    second(col(colhs)).alias("segundo_arribo")
)

print(nombre)

datos_tiempo_arribo.printSchema()

datos_tiempo_arribo.orderBy('Fecha_Arribo', ascending = True).show(5)

In [ ]:
#Estadisticos Descriptivos de algunas columnas de cada DataFrame
print(nombre)

print("Estadisticas Descriptivas de Retiros")
columnas = ["Genero_Usuario", "Edad_Usuario", "Bici", "Ciclo_Estacion_Retiro", "num_mes_retiro", "num_hora_retiro", "anio_retiro", "id"]
datos_tiempo_retiro.select(columnas).describe().show()

print("Estadisticas Descriptivas de Arribos")
columnas = ["Genero_Usuario", "Edad_Usuario", "Bici", "Ciclo_Estacion_Arribo", "num_mes_arribo", "num_hora_arribo", "anio_arribo", "id"]
datos_tiempo_arribo.select(columnas).describe().show()

## Debido a Limitantes con mi equipo, tuve que reducir las columnas que hacen las estadísticas descriptivas ya que
## se quedaba colgada con una carga de 100% de CPU y no entregaba ninguna salida. 
## (Lo dejé 10 minutos y no hizo nada, al reducirlas lo hizo en 2 mnutos).

In [ ]:
#Guardar el nuevo dataframe

print(nombre)

datos_tiempo_retiro.write.mode("overwrite").option("header", "true").csv("hdfs://namenode:9000/tmp/amd/ecobiciretiros")
print("Done")

datos_tiempo_arribo.write.mode("overwrite").option("header", "true").csv("hdfs://namenode:9000/tmp/amd/ecobiciarribos")
print("Done")